## Setting Up Jupyter Notebooks In The Amazon Cloud

If your computer is struggling with your MCMC calculations, you may want to get your hands on a more powerful computer. Amazon provides a service that allows you to use their computers on demand, and this brief tutorial will help you get your Jupyter Notebooks up and running in their computer cloud.

### 1. Create an Amazon Web Services account

The first thing to do is create an AWS account by clicking on the link on the upper right corner of this [page](https://aws.amazon.com/). Once you have your account you can go back to it by clicking on *Sign In To Console*, which should now be in the upper right corner of the same site.

### 2. Set up your Server

Now you can select the kind of computer you need. You will be using Amazon's [*Elastic Compute Cloud*](https://aws.amazon.com/ec2/) (EC2) service. To do this, in the menu located in the upper left part of your console, click on *Services* &rightarrow; *EC2*. This is the EC2 dashboard.

Within this service, there are two main things that you need to choose, essentialy the software and the hardware. You will do this by creating an *instance*, which you can do by clicking on the *Launch Instance* button under *Create Instance*. 

For your *Amazon Machine Image*, or your software configuration, select *Amazon Linux AMI (HVM / 64-bit)*.

For your *Instance Type*, or hardware configuration, select *c4.xlarge*. This will be around $0.2 per hour of usage. You can see more details about this [here](https://aws.amazon.com/ec2/pricing/on-demand/).

On the lower right corner, click on *Next: Configure Instance Details*. Leave the default options, and click on *Next: Add Storage*.

Change the size to 30 GiB, volume type should be the default General Purpose SSD (GP2). Click on *Next: Add Tags*. You do not need to change anything here. Click on *Next: Configure Security Group*

With the *Create new security group* option checked, click on the *Add Rule* button in the lower part. Fill in as follows:

| Type        | Protocol           | Port Range  | Source  |
| ------------- |:-------------:| -----:| -----:|
| Custom TCP      | TCP | 8888 | Anywhere |

This configuration should enable you to connect to your Jupyter Notebooks from any IP address.

On the lower right corner, click on *Review and Launch*. Again, on the lower right corner, click on *Launch*. 

In the popup window, select *Create a new key pair*, name it *bebi103_keypair*, and click on *Download Key Pair*. In your computer, move this file to your bebi103 directory.
Finally, click on *Launch Instance*.

Your have a server up and running!

Click on *View Instances* to see your server details. You will come back to this window later to retrieve your IP address.

### 3. Connect to your server

Now that you have a server, you can connect to it using your computer and the *ssh* protocol. On Mac or Linux, open a terminal window. On Windows, you will need to download and install an ssh client such as [PuTTY](http://www.putty.org). Navigate to your bebi103 directory, where you saved the bebi103_keypair file:

```
cd path_to_bebi103directory
```

Change permissions on your key file to make sure that it isn't publicly viewable: 

```
chmod 400 bebi103_keypair.pem.txt
```

Make your computer aware of this key for ssh connection to your server:

```
ssh-add -K bebi103_keypair.pem.txt
```

Now you can connect to the server using *ssh*. To find your server, you need its IP address, which you can find using your AWS console in your last opened web browser window (see below); you can also go back to this window by clicking on the left panel of your EC2 dashboard on *Instances*. 

To get your server's IP address, click on your BeBi103 instance. On the lower part, in upper right of the *Description* tab, you should see *IPv4 Public IP*. Copy this address, it should be something like: ``13.58.35.204``. The IP address will change when you restart your server, so make sure you to retrieve it from here next time you need it.

Go back to the terminal and connect to your server using your username (*ec2-user* is the generic name for the software configuration we selected) and your server's current IP address:

```
ssh ec2-user@Your.IP.Address
```

Type `yes` if asked to continue connecting. You should get a welcome message with ``Amazon Linux AMI`` on it, and your terminal should now show something like: 

```
[ec2-user@ip-172-31-24-56 ~]$
```

If this is the case, you are connected to your server! You can type `logout` whenever you want to do so. Anytime you want to connect again, all you need to do is fetch your server's IP address, open a terminal window and type the last command to connect using ssh, since your computer will remember your keypair.

### 4. Install Anaconda

You can install Anaconda with python3 on your server by typing the following commands in the terminal, while connected to the server via ssh:

```
wget https://repo.continuum.io/archive/Anaconda3-4.4.0-Linux-x86_64.sh
bash Anaconda3-4.2.0-Linux-x86_64.sh
```

Press enter a few times and type *yes* to agree; be sure to allow the installer to prepend the install location to your PATH. 

When done, you will need to refresh the *.bashrc* file to make the computer aware of your changes by typing this in the same terminal:

```
source .bashrc
```

Now your server should be using the python from anaconda. You can type the following command to find out:

```
which python
```

which should output:

```
/home/ubuntu/anaconda3/bin/python
```


You are all set to use Anaconda on your server, which also includes the conda package manager that you can use to install any other python packages you may need, such as `emcee`. 

You should do that by typing:

```
conda install -c astropy emcee
```

Then type *y* and enter to proceed. 

#### Installing github and the bebi103_utils repository

Enter the following while connected to the server via ssh:

```
sudo yum install git
git clone https://github.com/justinbois/bebi103_utils.git
```

Then modify your PYTHONPATH:
```
vim ~/.bashrc
```
Enter edit mode by pressing *O*, then copy and paste the following:
```
export PYTHONPATH=:.:$HOME/bin/PQpython_utils:$PYTHONPATH
```
Save and exit by pressing *:wq* and enter. Refresh your .bashrc: 
```
source ~/.bashrc
```

To fix a bug and be able to run bebi103:
```
sudo yum install mesa-libGL.x86_64
```
You should now be able to import bebi103_utils. You could already start computing on Amazon's cloud. But we will do one last configuration step so that you can access your Jupyter Notebooks from a web browser in your own computer.

### 5. Set up Jupyter Notebooks

While connected to the server via *ssh*, open an interactive python console by typing:

```
ipython
```

In the ipython console, type:
```
from IPython.lib import passwd
```

Generate a password:

```
passwd(yourpassword)
```

Which should print a string like the following:

```
'sha1:98ff0e580111:12798c72623a6eecd54b51c006b1050f0ac1a62d'
```

Copy and paste this hash somewhere in your computer and exit the `ipython` console by typing `exit` and enter.
Now you should let Jupyter know your password by typing the following in the terminal:

```
jupyter notebook --generate-config
mkdir certs
cd certs
sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem
```

Complete your certificate details and press enter after each one.

Now you have to do the following to access your Jupyter notebook configuration file and update it.


Replace the string between quotes in the password line below with your hash from earlier. Copy these lines, you will paste them in your Jupyter notebook configuration file.

```
c = get_config()
c.IPKernelApp.pylab = 'inline' 
c.NotebookApp.certfile = u'/home/ec2-user/certs/mycert.pem' 
c.NotebookApp.ip = '*' 
c.NotebookApp.open_browser = False
# Replace this hash with your own
c.NotebookApp.password = u'sha1:941c93244463:0a28b4a9a472da0dc28e79351660964ab81605ae' 
c.NotebookApp.port = 8888
```
Open the configuration file using the *vim* text editor:

```
vim ~/.jupyter/jupyter_notebook_config.py
```

To go into edit mode, press *O* (Capital letter O). Paste the lines from above containing your own password; you can do so with your regular paste command (e.g. cmd + p in mac).

Exit edit mode by pressing *Esc*, then save and exit vim by pressing *:wq* followed by enter.

Create a directory for your notebooks, your data and output, and open Jupyter:

```
cd
mkdir notebooks
cd notebooks
mkdir data
mkdir output
jupyter notebook&
```

Your Jupyter Notebooks should now be available through the internet. To access them, open a web browser, I recommend using Chrome. Enter the IP address of your server followed by *:8888*, which indicates the port we setup for Jupyter:

```
https://Your.IP.address:8888
```

If you see a security warning, bypass it. In Chrome, click on *Advanced* and proceed to your site.
Type in your password, the human-readable one that you used to generate the hash before, which was the argument of `passwd(yourpassword)`.

That's it, you should be able to access your Jupyter Notebooks running on Amazon's computers using a web browser! 
Notice that Jupyter has to be running on the server in order for you to be able to access it.

### Transfering files to and from your server

After your setup, transferring files to and from the server should be easy. With your server's IP address in hand, you can use *SCP* from your computer, without being connected to the server.

Open a new terminal window. To transfer files from your computer to your data directory on the server, enter the following:
```
scp path_to_file ec2-user@Your.IP.Address:/home/ec2-user/notebooks/data/file_name
```
You can also transfer directories by using the recursive option:
```
scp -r path_to_directory ec2-user@Your.IP.Address:/home/ec2-user/notebooks/data/directory_name
```
To transfer a file from the server data directory to your computer, you just invert the arguments. For example, to transfer your whole output directory:
```
scp -r ec2-user@Your.IP.Address:/home/ec2-user/notebooks/data/output path_to_directory
```

### Managing your server time

You will be charged for the time your server is running. When you are not using it, you can temporarily stop it so that you don't get charged. To do this, go to your *EC2* dashboard and click on *instances* on the left panel. 

Click on *Actions* (if you have more than one instance, right click on the instance you want to stop). 

Select *Instance State* &rightarrow; *Stop*. **Do not select terminate**, as this will delete your configuration. 

When you need to use it again, you can restart it using the same procedure, but instead select *Instance State* &rightarrow; *Start*. You do not need to be connected to the server for it to be running, but you have to have Jupyter running in order to access it. For example, you can connect to the server, start Jupyter, logout from the server, and keep using Jupyter through your web browser.

We have added a handy function to *bebi103_utils* that you can use to be notified by email when calculations are ready, so that you can stop the server. We also created a default gmail account that you can use for that. Simply add the following line in your python script, after your calculation code:

```
bebi103.send_notification('your_email_address', 'MCMCpower')
```

Replace the first argument with your email address (e.g. john@gmail.com), and you will receive an email notifying you once that line is run. 